#Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
First let's import libraries

In [105]:
import pandas as pd
import numpy as np

The html-site is assigned and loaded via pandas read_html. Afterward we will assign the first table on the html site under dfs-pandas dataframe.
Finally we are going to look up our 5 first values of our dfs-dataframe.

In [106]:
url='http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0, flavor='bs4')
dfs=pd.DataFrame(df[0])
dfs.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Let's exclude "Not assigned" Borough:

In [107]:
dfs=dfs[dfs.Borough != 'Not assigned']
dfs.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Let's join the duplicated Postcodes and split them within Neighbourhood-column with a comma. We specify not to order the dataframe:

In [108]:
dfs = dfs.groupby(['Postcode', 'Borough'], sort=False)['Neighbourhood'].agg([('Neighbourhood', ', '.join)])
dfs.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Not assigned


Next we are going to replace the "Not assigned" Neighbourhoods with the Borough-label and illustrate the dataframe:

In [113]:
dfs.at[dfs['Neighbourhood'] == 'Not assigned'] = "Queen's Park"
dfs.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront, Regent Park"
M6A,North York,"Lawrence Heights, Lawrence Manor"
M7A,Queen's Park,Queen's Park


The dataframe shape:

In [115]:
dfs.shape

(103, 1)